In [540]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [541]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [542]:
len(df_train['PassengerId'].unique())

711

In [543]:
df_train.describe()

,PassengerId,Age,SibSp,Parch,Fare
count,711.000000,573.000000,711.000000,711.000000,711.000000
mean,448.981716,28.845986,0.542897,0.393812,31.973669
std,257.156506,14.282324,1.112314,0.821998,50.468437
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.000000,19.000000,0.000000,0.000000,7.925000
50%,458.000000,28.000000,0.000000,0.000000,14.458300
75%,675.500000,36.500000,1.000000,0.000000,30.847900
max,891.000000,80.000000,8.000000,6.000000,512.329200


In [544]:
df_train.select_dtypes(include='object').describe()

,Survived,Pclass,Name,Sex,Ticket,Cabin,Embarked
count,711,711,711,711,711,158,711
unique,2,3,711,2,568,122,3
top,No,Lower,"Minahan, Miss. Daisy E",male,347082,G6,Southampton
freq,438,391,1,464,6,4,518


In [545]:
def procesar_df(df, num_title = 5):
    df_final = df[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']].set_index('PassengerId')
    df_final['Survived'] = df_final['Survived'].map({'No': 0, 'Yes': 1})
    df_final['Title'] = df_final['Name'].str.extract(r', (\w+)\.')

    idx = df_final['Title'].value_counts() >= num_title
    title_names = idx[idx].index.to_list()
    df_final['Title'] = df_final['Title'].apply(lambda x: x if x in title_names else 'Other')

    df_final = df_final.drop(['Name'], axis=1)

    df_final['Sex'] = df_final['Sex'].map({'female': 0, 'male':1})
    df_final['Pclass'] = df_final['Pclass'].map({'Lower': 3, 'Middle': 2, 'Upper': 1})
    df_final = pd.get_dummies(df_final, columns=['Embarked'])
    df_final = pd.get_dummies(df_final, columns=['Title'])

    df_final = df_final.dropna()

    return df_final

In [546]:
df_train = procesar_df(df_train, num_title = 5)
df_test = procesar_df(df_test, num_title=2)
df_test[['Title_Dr', 'Title_Rev']] = False

In [547]:
for col in ['SibSp', 'Parch', 'Fare', 'Pclass', 'Age']:
    max_ = df_train[col].max()
    min_ = df_train[col].min()

    df_train[col] = (df_train[col] - min_) / (max_ - min_) 
    df_test[col] = (df_test[col] - min_) / (max_ - min_) 

In [548]:
df_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived',
       'Embarked_Cherbourg', 'Embarked_Queenstown', 'Embarked_Southampton',
       'Title_Dr', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'Title_Other', 'Title_Rev'],
      dtype='object')

In [549]:
df_test.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived',
       'Embarked_Cherbourg', 'Embarked_Queenstown', 'Embarked_Southampton',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Other',
       'Title_Dr', 'Title_Rev'],
      dtype='object')

In [550]:
df_train.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
count,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000
mean,0.623909,0.636998,0.357200,0.108901,0.074171,0.067976,0.411867
std,0.415399,0.481285,0.179471,0.193625,0.145096,0.105472,0.492601
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.233476,0.000000,0.000000,0.015713,0.000000
50%,0.500000,1.000000,0.346569,0.000000,0.000000,0.030937,0.000000
75%,1.000000,1.000000,0.453380,0.200000,0.166667,0.064412,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [551]:
x_train = df_train.drop(['Survived'], axis=1).values
x_test = df_test.drop(['Survived'], axis=1).values

y_train = df_train['Survived'].values
y_test = df_test['Survived'].values

In [552]:
n_epochs = 500
batch_size = 128
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dropout(0.2, input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['AUC'])

hist = model.fit(x_train,
                 y_train,
                 shuffle = True,
                 batch_size= batch_size,
                 epochs=n_epochs,
                 validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).